In [ ]:
import sys
sys.path.append('../edaf')

In [ ]:
from plot_helpers import *
from decomp import *
from loguru import logger
import os, sys, gzip, json
import sqlite3
import pandas as pd
from edaf.core.uplink.preprocess import preprocess_ul
from edaf.core.uplink.analyze_packet import ULPacketAnalyzer
from edaf.core.uplink.analyze_channel import ULChannelAnalyzer
from edaf.core.uplink.analyze_scheduling import ULSchedulingAnalyzer
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from pathlib import Path
import ijson

%load_ext autoreload
%autoreload 2

In [ ]:
# Remove default handler
logger.remove()
# Add a new handler with level WARNING
logger.add(sys.stdout, level="ERROR")

In [ ]:
MEAS_DIR = '../meas-oai5g/'
MEAS_LABEL = 's19'
DB_FILE = MEAS_DIR+MEAS_LABEL+'/'+'res_db.sql'
PLOTS_DIR = './plots/'+MEAS_LABEL+'/'

In [ ]:
# packet analyzer
analyzer = ULPacketAnalyzer(DB_FILE)
uids_arr = range(analyzer.first_ueipid, analyzer.last_ueipid+1)
packets = analyzer.figure_packettx_from_ueipids(uids_arr)

# sched analyzer
ip_in_0_ts = packets[0]['ip.in_t']
sched_analyzer = ULSchedulingAnalyzer(
    total_prbs_num = 106, 
    symbols_per_slot = 14, 
    slots_per_frame = 20, 
    slots_duration_ms = 0.5, 
    scheduling_map_num_integers = 4,
    max_num_frames=100,
    db_addr = DB_FILE
)
begin_ts = ip_in_0_ts
end_ts = sched_analyzer.last_ts
logger.info("Scheduling events at GNB:")
sched_arr = sched_analyzer.find_resource_schedules_from_ts(begin_ts, end_ts)
logger.info("Buffer status updates:")
bsrupd_arr = sched_analyzer.find_bsr_upd_from_ts(begin_ts, end_ts)

In [ ]:
send_ts = analyzer.nlmt_df['timestamps.client.send.wall'].to_numpy()
send_ts = (send_ts)/1e6
receive_ts = analyzer.nlmt_df['timestamps.server.receive.wall'].to_numpy()
receive_ts = (receive_ts)/1e6
idt = (send_ts[1:-1]-send_ts[0:-2])
e2e_delays = receive_ts-send_ts
frame_alignment_delays = np.array(list({packet['id']: get_frame_alignment_delay(packet['ip.in_t'], bsrupd_arr, slots_per_frame=20, slots_duration_ms=0.5) for packet in packets if get_frame_alignment_delay(packet['ip.in_t'], bsrupd_arr, slots_per_frame=20, slots_duration_ms=0.5)!=None}.values()))
scheduling_delays = np.array(list({packet['id']: get_scheduling_delay(packet['ip.in_t'], sched_arr, slots_per_frame=20, slots_duration_ms=0.5) for packet in packets if get_scheduling_delay(packet['ip.in_t'], sched_arr, slots_per_frame=20, slots_duration_ms=0.5)!=None}.values()))
ran_delays = np.array(list({packet['id']: get_ran_delay(packet) for packet in packets if get_ran_delay(packet)!=None}.values()))
queueing_delays = np.array(list({packet['id']: get_queueing_delay(packet) for packet in packets if get_queueing_delay(packet)!=None}.values()))
retx_delays = np.array(list({packet['id']: get_retx_delay(packet) for packet in packets if get_retx_delay(packet)!=None}.values()))

In [ ]:
# skip first and last packets
SKIP_FIRST = 1000
SKIP_LAST = 100

fig, ax  = plot_ccdf(e2e_delays[SKIP_FIRST:-SKIP_LAST], '_', figsize=(8, 5))
plt.savefig(f"{PLOTS_DIR}e2e_delays_ccdf_plot.png", dpi=300, bbox_inches='tight')  # You can change the file format (e.g., .pdf)
plt.show()

fig, ax  = plot_ccdf(ran_delays[SKIP_FIRST:-SKIP_LAST], '_', figsize=(8, 5))
plt.savefig(f"{PLOTS_DIR}ran_delays_ccdf_plot.png", dpi=300, bbox_inches='tight')  # You can change the file format (e.g., .pdf)
plt.show()


fig, ax  = plot_ccdf(queueing_delays[SKIP_FIRST:-SKIP_LAST], '_', figsize=(8, 5))
plt.savefig(f"{PLOTS_DIR}queueing_delays_ccdf_plot.png", dpi=300, bbox_inches='tight')  # You can change the file format (e.g., .pdf)
plt.show()

fig, ax  = plot_ccdf(retx_delays[SKIP_FIRST:-SKIP_LAST], '_', figsize=(8, 5))
plt.savefig(f"{PLOTS_DIR}retx_delays_ccdf_plot.png", dpi=300, bbox_inches='tight')  # You can change the file format (e.g., .pdf)
plt.show()

In [ ]:
# plotting for frame alignment delay and scheduling delay
index_range = slice(1000, 1050)
SKIP_FIRST = 1000
SKIP_LAST = 100

# Create a figure and axes
fig, ax = plt.subplots(figsize=(12,5))

# Plotting each data set with different markers
ax.plot(frame_alignment_delays[index_range], marker='*', label='Frame alignment Delay')
ax.plot(scheduling_delays[index_range], marker='x', label='Scheduling delay')
ax.plot(ran_delays[index_range], marker='o', label='RAN delay')

# Adding labels and title
ax.set_xlabel('Index')
ax.set_ylabel('Delay [ms]')

# Adding grid
ax.grid(True)

# Adding legend
ax.legend()

# Save the plot
plt.savefig(f"{PLOTS_DIR}ran_delay_timeseries.png", dpi=300, bbox_inches='tight')

# Display the plot
plt.show()

## ran delay without frame alignment delay and scheduling
# Create a figure and axes
fig, ax = plt.subplots(figsize=(12,5))

# Plotting each data set with different markers
ax.plot(ran_delays[index_range]-frame_alignment_delays[index_range], marker='o', label='RAN Delay w/o frame-alignment delay')
ax.plot(ran_delays[index_range]-scheduling_delays[index_range], marker='o', label='RAN Delay w/o scheduling delau')

# Adding labels and title
ax.set_xlabel('Index')
ax.set_ylabel('Delay [ms]')

# Adding grid
ax.grid(True)

# Adding legend
ax.legend()

# Save the plot
plt.savefig(f"{PLOTS_DIR}ran_delay_wo_scheduling_timeseries.png", dpi=300, bbox_inches='tight')

## ccdf for ran delay without scheduling delay

fig, ax  = plot_ccdf(ran_delays[SKIP_FIRST:-SKIP_LAST]-scheduling_delays[SKIP_FIRST:-SKIP_LAST], '_', figsize=(8, 5))
plt.savefig(f"{PLOTS_DIR}ran_delays_ccdf_wo_scheduling_plot.png", dpi=300, bbox_inches='tight')  # You can change the file format (e.g., .pdf)
plt.show()